In [11]:
import random
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from scipy.stats import linregress

In [12]:
df = pd.read_csv('ReviewsEN.csv')

In [13]:
print("-1 :", df['sentiment'].value_counts()[-1])
print("0 :", df['sentiment'].value_counts()[0])
print("1 :", df['sentiment'].value_counts()[1])

-1 : 870
0 : 639
1 : 1518


In [14]:
# Replace values in pandas DataFrame.
df['sentiment'] = df['sentiment'].replace([1], 2)
df['sentiment'] = df['sentiment'].replace([0], 1)
df['sentiment'] = df['sentiment'].replace([-1], 0)

## Case Folding

In [15]:
# Apply lower function
df['reviews'] = df['reviews'].apply(str.lower)
print(df)

      sentiment                                            reviews
0             1  teacher are punctual but they should also give...
1             2                                               good
2             2  excellent lectures are delivered by teachers a...
3             2  teachers give us all the information required ...
4             2                                                yes
...         ...                                                ...
3022          2        lecturers provide clear enough explanations
3023          2            lecturer's assessment is very objective
3024          0              lecturers give very good explanations
3025          0        lecturers often provoke discussion in class
3026          2         lecturers provide material in a boring way

[3027 rows x 2 columns]


## Hyper Parameter

In [16]:
# Global Variables

EMBEDDING_DIM = 100
MAXLEN = 16
TRUNCATING = 'post'
PADDING = 'post'
OOV_TOKEN = "<OOV>"
MAX_EXAMPLES = len(df)
TRAINING_SPLIT = 0.9

## Remove Punctuation

In [17]:
import string

def remove_punctuation(text):
    # Remove punctuation marks
    translator = str.maketrans('', '', string.punctuation)
    text_without_punctuation = text.translate(translator)
    return text_without_punctuation

In [18]:
removedPunctuation_text = []
for i in range(len(df)):
  removedPunctuation_text.append(remove_punctuation(df['reviews'][i]))

df['reviews'] = removedPunctuation_text
df['reviews'].values

array(['teacher are punctual but they should also give us the some practical knowledge other than theortical',
       'good',
       'excellent lectures are delivered by teachers and all teachers are very punctual',
       ..., 'lecturers give very good explanations',
       'lecturers often provoke discussion in class',
       'lecturers provide material in a boring way'], dtype=object)

## Lemmatizing

In [19]:
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')  
nltk.download('punkt')  

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = []
    tokens = nltk.word_tokenize(text)  
    for token in tokens:
        lemma = lemmatizer.lemmatize(token) 
        lemmatized_words.append(lemma)
    lemmatized_text = ' '.join(lemmatized_words)  
    return lemmatized_text


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [20]:
temp =  []
for i in range(len(df)):
  temp.append(lemmatize_text(df['reviews'][i]))
df['reviews'] = temp

## Convert Numbers

In [21]:
import re

def remove_numbers(text):
    cleaned_text = re.sub(r'\d+', '', text)  
    return cleaned_text

In [22]:
df['reviews'] = df['reviews'].apply(remove_numbers)
df

,sentiment,reviews
0,1,teacher are punctual but they should also give...
1,2,good
2,2,excellent lecture are delivered by teacher and...
3,2,teacher give u all the information required to...
4,2,yes
...,...,...
3022,2,lecturer provide clear enough explanation
3023,2,lecturer assessment is very objective
3024,0,lecturer give very good explanation
3025,0,lecturer often provoke discussion in class


## Random Sampling

In [23]:
import random

random.seed(42)

# Get the indices of the DataFrame
indices = df.index.tolist()

# Perform random sampling on the indices
selected_indices = random.sample(indices, MAX_EXAMPLES)

# Select the corresponding sentences and labels based on the sampled indices
sentences = df.loc[selected_indices, 'reviews']
labels = df.loc[selected_indices, 'sentiment']

print(f"There are {len(sentences)} sentences and {len(labels)} labels after random sampling\n")


There are 3027 sentences and 3027 labels after random sampling



# Training - Validation Split

In [24]:
def train_val_split(sentences, labels, training_split):
    ### START CODE HERE
    
    # Compute the number of sentences that will be used for training (should be an integer)
    train_size = int(len(sentences)*training_split)

    # Split the sentences and labels into train/validation splits
    train_sentences = sentences[:train_size]
    train_labels = labels[:train_size]

    test_sentences = sentences[train_size:]
    test_labels = labels[train_size:]
    
    ### END CODE HERE
    
    return train_sentences, test_sentences, train_labels, test_labels

In [25]:
train_sentences, test_sentences, train_labels, test_labels = train_val_split(sentences, labels, TRAINING_SPLIT)

print(f"There are {len(train_sentences)} sentences for training.\n")
print(f"There are {len(train_labels)} labels for training.\n")
print(f"There are {len(test_sentences)} sentences for validation.\n")
print(f"There are {len(test_labels)} labels for validation.")

There are 2724 sentences for training.

There are 2724 labels for training.

There are 303 sentences for validation.

There are 303 labels for validation.


# Tokenization & Stopwords - Sequences, Truncating, and Padding

In [26]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### Stopwords

In [27]:
my_file = open("stopwords.txt", "r")
  
data = my_file.read()
  
stopwords_data = data.split("\n")
print(stopwords_data)
my_file.close()

['a', 'about', 'above', 'across', 'after', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'among', 'an', 'and', 'another', 'any', 'anybody', 'anyone', 'anything', 'anywhere', 'are', 'area', 'areas', 'around', 'as', 'ask', 'asked', 'asking', 'asks', 'at', 'away', 'b', 'back', 'backed', 'backing', 'backs', 'be', 'became', 'because', 'become', 'becomes', 'been', 'before', 'began', 'behind', 'being', 'beings', 'best', 'better', 'between', 'big', 'both', 'but', 'by', 'c', 'came', 'can', 'cannot', 'case', 'cases', 'certain', 'certainly', 'clear', 'clearly', 'come', 'could', 'd', 'did', 'differ', 'different', 'differently', 'do', 'does', 'done', 'down', 'down', 'downed', 'downing', 'downs', 'during', 'e', 'each', 'early', 'either', 'end', 'ended', 'ending', 'ends', 'enough', 'even', 'evenly', 'ever', 'every', 'everybody', 'everyone', 'everything', 'everywhere', 'f', 'face', 'faces', 'fact', 'facts', 'far', 'felt', 'few', 'find', 'finds', 'first

In [28]:
def remove_stopwords(sentence, data):
    # List of stopwords

    stopwords = data + ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
    numbers_stopwords = ["1", "2", "3", "4","5","6","7","8","9","10",
                         "one", "two","three","four","five","098"]
    more_words = ["didn't", "don't", "dont", "didnt", "it", "doesnt", "doesn't", "hw","won't","lpu","weren't","mr","mcq","shes",
                  "shes","india","in","hes","shes","me", "dr", "nlandu", "ko","it","1st", "omr", "ha", "upto","ca", "soo", "cd", "ive","po","cse", "chem", "un","of",
                  "mte", "omr","mte's","ca's","ete's","jnv","ip","sir","its","wks","prob","python","java","lattc","ol","ived","elsewhere", "mother","wouldnt","car",
                  "si", "sat","we","home","hot","god","ice","money's","money","even","about","thats", "wks", "thurs", "months", "sir", "go", "jnv", "ip", "today", "today's", "linux", "github",
                  "lt", "ums", "superb", "at", "cgpa","ques", "brain's", "mcqs", "ve", "say", "pc", "viva", "after", "before", "draw", "asst", "only", "rich", "never", "went", "pcs", "gk", "one's",
                  "co", "duty", "gona", "attendnce","same", "that's", "hahahah", "ad's", "university's", "relly", "build", "cricket", "said", "hall", "profs", "guy's", "can", "along", "archieve", "bag",
                  "part", "master", "push", "or", "add", "were", "virginia","human", "bless", "clean", "count", "onlineopen", "ounce", "brushing", "zero", "mail", "fys", "lowell", "stets", "untill", "until",
                  "prep", "appears", "giulia", "yuk", "memo", "ton", "110q", "unit", "80","re","by","order","fob", "sit", "from","art", "org", "4d", "3d", "cinema", "iii", "cal", "both", "sundays", "todays", "ad",
                  "yoursel","yourself", "kiss", "it'll", "obayani's", "anal", "pgs", "csci", "hw", "more", "able", "lecturer", "lecturer's", "student", "stundet's", "it", "want", "you","he's", "she's"]
    more =  [
    'a', 'about', 'above', 'after', 'again', 'against', "ain't", 'all', 'am', 'an', 'and', 'any', 'are', 'aren\'t', 'as', 
    'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'can\'t', 'cannot', 
    'could', 'couldn\'t', 'did', 'didn\'t', 'do', 'does', 'doesn\'t', 'doing', 'don\'t', 'down', 'during', 'each', 'few', 
    'for', 'from', 'further', 'had', 'hadn\'t', 'has', 'hasn\'t', 'have', 'haven\'t', 'having', 'he', 'he\'d', 'he\'ll', 
    'he\'s', 'her', 'here', 'here\'s', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'how\'s', 'i', 'i\'d', 'i\'ll', 
    'i\'m', 'i\'ve', 'if', 'in', 'into', 'is', 'isn\'t', 'it', 'it\'s', 'its', 'itself', 'let\'s', 'me', 'more', 'most', 
    'mustn\'t', 'my', 'myself', 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 
    'ours', 'ourselves', 'out', 'over', 'own', 'same', 'shan\'t', 'she', 'she\'d', 'she\'ll', 'she\'s', 'should', 
    'shouldn\'t', 'so', 'some', 'such', 'than', 'that', 'that\'s', 'the', 'their', 'theirs', 'them', 'themselves', 
    'then', 'there', 'there\'s', 'these', 'they', 'they\'d', 'they\'ll', 'they\'re', 'they\'ve', 'this', 'those', 
    'through', 'to', 'too', 'under', 'until', 'up', 'very', 'was', 'wasn\'t', 'we', 'we\'d', 'we\'ll', 'we\'re', 
    'we\'ve', 'were', 'weren\'t', 'what', 'what\'s', 'when', 'when\'s', 'where', 'where\'s', 'which', 'while', 'who', 
    'who\'s', 'whom', 'why', 'why\'s', 'with', 'won\'t', 'would', 'wouldn\'t', 'you', 'you\'d', 'you\'ll', 'you\'re', 
    'you\'ve', 'your', 'yours', 'yourself', 'yourselves']

    final_stopwords = stopwords + numbers_stopwords + more_words + more

    words = sentence.split()
    tempWords = []
    for i in words:
        if i not in final_stopwords:
            tempWords.append(i)
            sentence = ' '.join(tempWords)
    

    return sentence

In [29]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')  
nltk.download('stopwords')  

stop_words = set(stopwords.words('english'))
stop_words_list = list(stop_words)
stopwords_data = stopwords_data + stop_words_list

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english')  
response = vectorizer.fit_transform(df['reviews'])
print(type(response))

<class 'scipy.sparse._csr.csr_matrix'>


In [31]:
cKomen = []
for i in range(len(df)):
  cKomen.append(remove_stopwords(df['reviews'][i], stopwords_data))

df['reviews'] = cKomen

In [32]:
# GRADED FUNCTION: fit_tokenizer
def fit_tokenizer(sentences, oov_token):
    ### START CODE HERE
    
    # Instantiate the Tokenizer class, passing in the correct values for oov_token
    tokenizer = Tokenizer(oov_token = OOV_TOKEN)
    
    # Fit the tokenizer to the training sentences
    tokenizer.fit_on_texts(sentences)
    
    ### END CODE HERE
    
    return tokenizer

In [33]:
# Test your function
tokenizer = fit_tokenizer(train_sentences, OOV_TOKEN)
word_index = tokenizer.word_index
VOCAB_SIZE = len(word_index)

print(f"Vocabulary contains {VOCAB_SIZE} words\n")
print("<OOV> token included in vocabulary" if "<OOV>" in word_index else "<OOV> token NOT included in vocabulary")
print(f"\nindex of word 'i' should be {word_index['i']}")

Vocabulary contains 3669 words

<OOV> token included in vocabulary

index of word 'i' should be 10


In [34]:
def seq_pad_and_trunc(sentences, tokenizer, padding, truncating, maxlen):

    sequences = tokenizer.texts_to_sequences(sentences)
    pad_trunc_sequences = pad_sequences(sequences, maxlen= MAXLEN, padding = PADDING, truncating = TRUNCATING)
    return pad_trunc_sequences

In [35]:
train_pad_trunc_seq = seq_pad_and_trunc(train_sentences, tokenizer, PADDING, TRUNCATING, MAXLEN)
val_pad_trunc_seq = seq_pad_and_trunc(test_sentences, tokenizer, PADDING, TRUNCATING, MAXLEN)

print(f"Padded and truncated training sequences have shape: {train_pad_trunc_seq.shape}\n")
print(f"Padded and truncated validation sequences have shape: {val_pad_trunc_seq.shape}")

Padded and truncated training sequences have shape: (2724, 16)

Padded and truncated validation sequences have shape: (303, 16)


In [36]:
train_labels = np.array(train_labels)
val_labels = np.array(test_labels)

# Using pre-defined Embeddings

In [37]:
# Define path to file containing the embeddings
GLOVE_FILE = 'glove.6B.100d.txt'

# Initialize an empty embeddings index dictionary
GLOVE_EMBEDDINGS = {}

# Read file and fill GLOVE_EMBEDDINGS with its contents
with open(GLOVE_FILE) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        GLOVE_EMBEDDINGS[word] = coefs

# Represent the words in your vocabulary using the embeddings

In [38]:
# Initialize an empty numpy array with the appropriate size
EMBEDDINGS_MATRIX = np.zeros((VOCAB_SIZE+1, EMBEDDING_DIM))

# Iterate all of the words in the vocabulary and if the vector representation for 
# each word exists within GloVe's representations, save it in the EMBEDDINGS_MATRIX array
for word, i in word_index.items():
    embedding_vector = GLOVE_EMBEDDINGS.get(word)
    if embedding_vector is not None:
        EMBEDDINGS_MATRIX[i] = embedding_vector

# Define a model that does not overfit

Model with 0.001 learning rate

In [39]:
# def create_model(vocab_size, embedding_dim, maxlen, embeddings_matrix):
#     model = tf.keras.Sequential([ 
#         # This is how you need to set the Embedding layer when using pre-trained embeddings
#         tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=maxlen, weights=[embeddings_matrix], trainable=False),
#         tf.keras.layers.Conv1D(32, 5, activation='relu'),
#         tf.keras.layers.GlobalMaxPooling1D(),
#         tf.keras.layers.Dropout(0.2),
#         tf.keras.layers.Dense(32, activation='relu'),
#         tf.keras.layers.Dense(3, activation='softmax'),
#     ])
    
#     model.compile(loss='sparse_categorical_crossentropy',
#                   optimizer='adam',
#                   metrics=['accuracy']) 
#     return model

Model with 0.002 learning rate

In [40]:
from tensorflow.keras import optimizers

def create_model(vocab_size, embedding_dim, maxlen, embeddings_matrix):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=maxlen, weights=[embeddings_matrix], trainable=False),
        tf.keras.layers.Conv1D(64, 3, activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(3, activation='softmax'),
    ])
    optimizer = optimizers.Adam(learning_rate = 0.002)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy']) 
    return model


Model with 0.0025 learning rate

In [41]:
# from tensorflow.keras import optimizers

# def create_model(vocab_size, embedding_dim, maxlen, embeddings_matrix):
#     model = tf.keras.Sequential([
#         tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=maxlen, weights=[embeddings_matrix], trainable=False),
#         tf.keras.layers.Conv1D(64, 3, activation='relu'),
#         tf.keras.layers.MaxPooling1D(pool_size=2),
#         tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True)),
#         tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
#         tf.keras.layers.Dense(64, activation='relu'),
#         tf.keras.layers.Dropout(0.2),
#         tf.keras.layers.Dense(3, activation='softmax'),
#     ])
#     optimizer = optimizers.Adam(learning_rate = 0.0025)
#     model.compile(loss='sparse_categorical_crossentropy',
#                   optimizer=optimizer,
#                   metrics=['accuracy']) 
#     return model


In [42]:
model = create_model(VOCAB_SIZE, EMBEDDING_DIM, MAXLEN, EMBEDDINGS_MATRIX)

In [43]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 16, 100)           367000    
                                                                 
 conv1d (Conv1D)             (None, 14, 64)            19264     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 7, 64)            0         
 )                                                               
                                                                 
 bidirectional (Bidirectiona  (None, 7, 128)           49920     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               31104     
 nal)                                                            
                                                        

In [44]:
# Train the model and save the training history
history = model.fit(train_pad_trunc_seq, train_labels, epochs=200, validation_data=(val_pad_trunc_seq, val_labels))

Epoch 1/200
86/86 [==============================] - 30s 42ms/step - loss: 0.9153 - accuracy: 0.5778 - val_loss: 0.7652 - val_accuracy: 0.6733
Epoch 2/200
86/86 [==============================] - 2s 24ms/step - loss: 0.7061 - accuracy: 0.7107 - val_loss: 0.6655 - val_accuracy: 0.7360
Epoch 3/200
86/86 [==============================] - 2s 19ms/step - loss: 0.5716 - accuracy: 0.7794 - val_loss: 0.6788 - val_accuracy: 0.7426
Epoch 4/200
86/86 [==============================] - 1s 16ms/step - loss: 0.5143 - accuracy: 0.7981 - val_loss: 0.7477 - val_accuracy: 0.7129
Epoch 5/200
86/86 [==============================] - 1s 17ms/step - loss: 0.3905 - accuracy: 0.8550 - val_loss: 0.7317 - val_accuracy: 0.7393
Epoch 6/200
86/86 [==============================] - 2s 20ms/step - loss: 0.3174 - accuracy: 0.8814 - val_loss: 0.7990 - val_accuracy: 0.7360
Epoch 7/200
86/86 [==============================] - 2s 19ms/step - loss: 0.2657 - accuracy: 0.9053 - val_loss: 0.9699 - val_accuracy: 0.6700
Epoch

In [45]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Make predictions on the validation set
val_predictions = model.predict(val_pad_trunc_seq)
val_predicted_labels = np.argmax(val_predictions, axis=1)

# Calculate precision, recall, and F1-score
precision = precision_score(val_labels, val_predicted_labels, average='weighted')
recall = recall_score(val_labels, val_predicted_labels, average='weighted')
f1 = f1_score(val_labels, val_predicted_labels, average='weighted')

print("Precision: ", precision)
print("Recall: ", recall)
print("F1-score: ", f1)


10/10 [==============================] - 1s 4ms/step
Precision:  0.7647184073246034
Recall:  0.7656765676567657
F1-score:  0.7649426279837286


Based on the evaluation metrics, our model performs relatively well with relatively high values of precision, recall, and F1-score

# Testing

In [57]:
# Assuming you have trained the model and obtained the history

# Random text for prediction
random_text = "Smart , but can't teach"

# Tokenize the random text
random_text_sequence = tokenizer.texts_to_sequences([random_text])

# Pad and truncate the sequence
random_text_sequence = pad_sequences(random_text_sequence, maxlen=MAXLEN, padding=PADDING, truncating=TRUNCATING)

# Make prediction
prediction = model.predict(random_text_sequence)[0]

# Convert prediction to sentiment label
predicted_label = np.argmax(prediction)

# Map sentiment label to sentiment interpretation
sentiment = "Positive" if predicted_label == 2 else "Neutral" if predicted_label == 1 else "Negative"

# Print the prediction result
print(f"Text: {random_text}")
print(f"Predicted Sentiment: {sentiment}")


1/1 [==============================] - 0s 32ms/step
Text: Smart , but can't teach
Predicted Sentiment: Neutral


In [58]:
# Random text for prediction
random_text = "Bad at teaching"

# Tokenize the random text
random_text_sequence = tokenizer.texts_to_sequences([random_text])
random_text_sequence = pad_sequences(random_text_sequence, maxlen=MAXLEN, padding=PADDING, truncating=TRUNCATING)
random_text_sequence

array([[152,  61,  62,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0]], dtype=int32)

In [59]:
prediction = model.predict(random_text_sequence)[0]

1/1 [==============================] - 0s 20ms/step


In [65]:
prediction
# Convert prediction to sentiment label
predicted_label = np.argmax(prediction)

# Map sentiment label to sentiment interpretation
sentiment = "Positive" if predicted_label == 2 else "Neutral" if predicted_label == 1 else "Negative"

In [66]:
sentiment

'Negative'

# Deployment

In [62]:
# import tensorflow as tf
# from tensorflow import keras

In [63]:
# tf.saved_model.save(model, 'saved_model')
# model.save('model.h5')



In [64]:
# converter = tf.lite.TFLiteConverter.from_saved_model('saved_model')
# tflite_model = converter.convert()
# with open('model.tflite', 'wb') as f:
#     f.write(tflite_model)